# Step 1: Foundational Patterns - Executors and Edges

## Overview

This notebook demonstrates the foundational concepts of the Agent Framework workflow system:

### Key Concepts Covered:

1. **Two Ways to Define Executors:**
   - Custom class subclassing `Executor` with `@handler` decorated methods
   - Standalone async functions decorated with `@executor`

2. **Handler Signatures:**
   - First parameter: typed input to the node
   - Second parameter: `WorkflowContext[T_Out, T_W_Out]`
     - `T_Out`: type sent to downstream nodes via `ctx.send_message()`
     - `T_W_Out`: type yielded as workflow output via `ctx.yield_output()`

3. **Workflow Builder:**
   - Fluent API: `add_edge()`, `set_start_executor()`, `build()`
   - Connect nodes with directed edges

4. **Running Workflows:**
   - `workflow.run(initial_input)` executes the graph
   - Terminal nodes yield outputs using `ctx.yield_output()`

### Prerequisites:
- No external services required
- Basic understanding of async/await in Python

## Import Required Libraries

In [ ]:
import os
from dotenv import load_dotenv
import asyncio

from agent_framework import (
    Executor,
    WorkflowBuilder,
    WorkflowContext,
    executor,
    handler,
)
from typing_extensions import Never

load_dotenv('../../.env')

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")


## Example 1: Custom Executor Subclass

### UpperCase Executor

Subclassing `Executor` allows you to define a named node with lifecycle hooks if needed. The work is implemented in an async method decorated with `@handler`.

**Handler Signature Contract:**
- **First parameter**: Typed input to this node (here: `text: str`)
- **Second parameter**: `WorkflowContext[T_Out]` where `T_Out` is the type emitted via `ctx.send_message()`

**Typical Handler Flow:**
1. Compute a result
2. Forward result to downstream nodes using `ctx.send_message(result)`

In [2]:
class UpperCase(Executor):
    def __init__(self, id: str):
        super().__init__(id=id)

    @handler
    async def to_upper_case(self, text: str, ctx: WorkflowContext[str]) -> None:
        """Convert the input to uppercase and forward it to the next node.

        Note: The WorkflowContext is parameterized with the type this handler will
        emit. Here WorkflowContext[str] means downstream nodes should expect str.
        """
        result = text.upper()

        # Send the result to the next executor in the workflow.
        await ctx.send_message(result)

## Example 2: Standalone Function-Based Executor

### ReverseText Executor

For simple steps, you can skip subclassing and define an async function with the same signature pattern, decorated with `@executor`.

**WorkflowContext Parameters:**
- `T_Out = Never`: This node does NOT send messages to downstream nodes
- `T_W_Out = str`: This node yields workflow output of type `str`

This is a **terminal node** that yields the final output using `ctx.yield_output()`.

In [3]:
@executor(id="reverse_text_executor")
async def reverse_text(text: str, ctx: WorkflowContext[Never, str]) -> None:
    """Reverse the input string and yield the workflow output.

    This node yields the final output using ctx.yield_output(result).
    The workflow will complete when it becomes idle (no more work to do).

    The WorkflowContext is parameterized with two types:
    - T_Out = Never: this node does not send messages to downstream nodes.
    - T_W_Out = str: this node yields workflow output of type str.
    """
    result = text[::-1]

    # Yield the output - the workflow will complete when idle
    await ctx.yield_output(result)

## Build and Run the Workflow

### Workflow Construction

Using the fluent `WorkflowBuilder` API:
1. `add_edge(from_node, to_node)` - defines directed edge: upper_case → reverse_text
2. `set_start_executor(node)` - declares the entry point
3. `build()` - finalizes and returns an immutable Workflow object

In [4]:
# Create executor instances
upper_case = UpperCase(id="upper_case_executor")

# Build the workflow using fluent pattern
workflow = (
    WorkflowBuilder()
    .add_edge(upper_case, reverse_text)
    .set_start_executor(upper_case)
    .build()
)

print("✅ Workflow built successfully!")

✅ Workflow built successfully!


## Execute the Workflow

### Running the Workflow

- `workflow.run(initial_input)` - executes the graph with initial input
- Returns an event collection
- `get_outputs()` - retrieves outputs yielded by terminal nodes
- `get_final_state()` - shows the final workflow state

In [5]:
# Run the workflow
events = await workflow.run("hello world")

# Get the outputs
outputs = events.get_outputs()
print("\n📤 Workflow Outputs:")
print(outputs)

# Get final state
final_state = events.get_final_state()
print(f"\n✅ Final state: {final_state}")


📤 Workflow Outputs:
['DLROW OLLEH']

✅ Final state: WorkflowRunState.IDLE


## Expected Output

```
['DLROW OLLEH']
Final state: WorkflowRunState.COMPLETED
```

### How It Works:

1. **Input**: `"hello world"`
2. **UpperCase Executor**: Converts to `"HELLO WORLD"` and sends to next node
3. **ReverseText Executor**: Reverses to `"DLROW OLLEH"` and yields as output
4. **Workflow completes** when all nodes are idle

## Key Takeaways

### Executor Patterns

✅ **Custom Class Executor** (UpperCase)
- Use when you need lifecycle hooks or complex state management
- Subclass `Executor` and use `@handler` decorator
- Best for reusable, stateful components

✅ **Function-Based Executor** (reverse_text)
- Use for simple, stateless transformations
- Just decorate with `@executor(id="...")`
- More concise for simple operations

### WorkflowContext Types

- `WorkflowContext[str]` - sends `str` to downstream nodes
- `WorkflowContext[Never, str]` - yields `str` as workflow output (terminal node)
- `WorkflowContext` - neither sends nor yields (equivalent to `WorkflowContext[Never, Never]`)

### Workflow Building

1. Create executor instances
2. Use `WorkflowBuilder()` fluent API
3. Connect with `add_edge(from, to)`
4. Set entry point with `set_start_executor()`
5. Finalize with `build()`

### Next Steps

Continue to **Step 2** to learn how to integrate AI agents into workflows!